# General Imports

    !! IMPORTANT !!
    If you did NOT install opengrid with pip, 
    make sure the path to the opengrid folder is added to your PYTHONPATH

In [1]:
import os
import inspect
import sys
import pandas as pd
import charts

from opengrid_dev.library import houseprint

Server running in the folder /usr/local/opengrid/notebooks/Demo at 127.0.0.1:52141


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 16,8

/usr/lib/python3/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/usr/lib/python3/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


## Houseprint

In [3]:
hp = houseprint.Houseprint()
# for testing:
# hp = houseprint.Houseprint(spreadsheet='unit and integration test houseprint')

Opening connection to Houseprint sheet
Opening spreadsheets
Parsing spreadsheets
35 Sites created
36 Devices created
102 sensors created
Houseprint parsing complete


In [4]:
hp


    Houseprint
    Created on 2017-10-05 14:30:21.476466 (UTC)
    35 sites
    36 devices
    102 sensors
    

In [5]:
hp.sites[:5]

[
     Site
     Key: 1
     1 devices
     6 sensors
     , 
     Site
     Key: 2
     1 devices
     6 sensors
     , 
     Site
     Key: 3
     1 devices
     1 sensors
     , 
     Site
     Key: 4
     1 devices
     4 sensors
     , 
     Site
     Key: 5
     1 devices
     1 sensors
     ]

In [6]:
hp.get_devices()[:4]

[
     Fluksometer
     Key: FL03001552
     6 sensors
     , 
     Fluksometer
     Key: FL03001561
     6 sensors
     , 
     Fluksometer
     Key: FL02000449
     1 sensors
     , 
     Fluksometer
     Key: FL03001566
     4 sensors
     ]

In [7]:
hp.get_sensors('water')[:3]

[
     Fluksosensor
     Key: 661684d35cd07dfc657a60b844860785
     Type: water
     , 
     Fluksosensor
     Key: b28509eb97137e723995838c393d49df
     Type: water
     , 
     Fluksosensor
     Key: 36dac700f81c6be9ca233e6534182ed4
     Type: water
     ]

A Houseprint object can be saved as a pickle. It loses its tmpo session however (connections cannot be pickled)

In [8]:
hp.save('new_houseprint.pkl')

Saved houseprint to /usr/local/opengrid/notebooks/Demo/new_houseprint.pkl


In [9]:
hp = houseprint.load_houseprint_from_file('new_houseprint.pkl')

### TMPO

The houseprint, sites, devices and sensors all have a get_data method. In order to get these working for the fluksosensors, the houseprint creates a tmpo session.

In [10]:
hp.init_tmpo()
hp._tmpos.debug = False
hp.sync_tmpos()

  0%|          | 0/102 [00:00<?, ?it/s]

Using tmpo database from /data/.tmpo/tmpo.sqlite3


100%|██████████| 102/102 [00:17<00:00,  5.94it/s]
/usr/lib/python3.5/json/encoder.py:198: DeprecationWarning: Interpreting naive datetime as local 2017-10-05 14:30:29.702494. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


## Lookup sites, devices, sensors based on key

These methods return a single object

In [ ]:
hp.find_site(1)

In [ ]:
hp.find_device('FL03001562')

In [ ]:
sensor = hp.find_sensor('d5a747b86224834f745f4c9775d70241')

In [ ]:
print(sensor.site)
print(sensor.unit)

## Lookup sites, devices, sensors based on search criteria

These methods return a list with objects satisfying the criteria

In [ ]:
hp.search_sites(inhabitants=5)

In [ ]:
hp.search_sensors(type='electricity', direction='Import')

### Get Data

In [11]:
hp.sync_tmpos()

100%|██████████| 102/102 [00:05<00:00, 14.71it/s]
/usr/lib/python3.5/json/encoder.py:198: DeprecationWarning: Interpreting naive datetime as local 2017-10-05 14:36:40.823000. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [12]:
head = pd.Timestamp('20150101')
tail = pd.Timestamp('20160101')
df = hp.get_data(sensortype='water', head=head, tail=tail, diff=True, resample='min', unit='l/min')
#charts.plot(df, stock=True, show='inline')

/usr/lib/python3.5/json/encoder.py:198: DeprecationWarning: Interpreting naive datetime as local 2017-10-05 14:37:14.457681. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 525580 entries, 2015-01-01 00:17:00+00:00 to 2015-12-31 23:56:00+00:00
Freq: T
Data columns (total 10 columns):
661684d35cd07dfc657a60b844860785    524345 non-null float64
b28509eb97137e723995838c393d49df    525578 non-null float64
2923b75daf93e539e37ce5177c0008c5    523283 non-null float64
a8054ca7865584e69a7c946aeea1e13c    525510 non-null float64
1a5c55ec176d24483572aca420ac59b5    511567 non-null float64
c57f45b21e2ec21ab427f172478e1ca4    389076 non-null float64
a926bc966f178fc5d507a569a5bfc3d7    525559 non-null float64
a519ce042e8748570d3a91cfca16a747    386143 non-null float64
25b8db566e6d930d4a6ae09e85cdcc1b    183800 non-null float64
5058b696beda910047f547fc4336ac17    225495 non-null float64
dtypes: float64(10)
memory usage: 44.1 MB


/usr/lib/python3.5/json/encoder.py:198: DeprecationWarning: Interpreting naive datetime as local 2017-10-05 14:37:27.239310. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


## Site

In [ ]:
site = hp.find_site(1)
site

In [ ]:
print(site.size)
print(site.inhabitants)
print(site.postcode)
print(site.construction_year)
print(site.k_level)
print(site.e_level)
print(site.epc_cert)

In [ ]:
site.devices

In [ ]:
site.get_sensors('electricity')

In [ ]:
head = pd.Timestamp('20150617')
tail = pd.Timestamp('20150628')
df=site.get_data(sensortype='electricity', head=head,tail=tail, diff=True, unit='kW')
charts.plot(df, stock=True, show='inline')

## Device

In [ ]:
device = hp.find_device('FL03001552')
device

In [ ]:
device.key

In [ ]:
device.get_sensors('gas')

In [ ]:
head = pd.Timestamp('20151101')
tail = pd.Timestamp('20151104')
df = hp.get_data(sensortype='gas', head=head,tail=tail, diff=True, unit='kW')
charts.plot(df, stock=True, show='inline')

## Sensor

In [ ]:
sensor = hp.find_sensor('53b1eb0479c83dee927fff10b0cb0fe6')
sensor

In [ ]:
sensor.key

In [ ]:
sensor.type

In [ ]:
sensor.description

In [ ]:
sensor.system

In [ ]:
sensor.unit

In [ ]:
head = pd.Timestamp('20150617')
tail = pd.Timestamp('20150618')
df=sensor.get_data(head,tail,diff=True, unit='W')
charts.plot(df, stock=True, show='inline')

## Getting data for a selection of sensors

In [ ]:
sensors = hp.search_sensors(type='electricity', system='solar')
print(sensors)
df = hp.get_data(sensors=sensors, head=head, tail=tail, diff=True, unit='W')
charts.plot(df, stock=True, show='inline')

## Dynamically loading data sensor per sensor

A call to `hp.get_data()` is lazy: it creates a big list of Data Series per sensor and concatenates them. This can take a while, specifically when you need many sensors and a large time span.

Often, you don't use the big DataFrame as a whole, you rather re-divide it by using a for loop and looking at each sensor individually.

By using `hp.get_data_dynamic()`, data is fetched from tmpo per sensor at a time, just when you need it.

In [ ]:
dyn_data = hp.get_data_dynamic(sensortype='electricity', head=head, tail=tail)

In [ ]:
ts = next(dyn_data)
df = pd.DataFrame(ts)
charts.plot(df, stock=True, show='inline')

You can run the cell above multiple times and eacht time the next sensor will be fetched